# CSC5651 - Applied Project - Detecting soccer players in broadcast footage - Notebook

This notebook contains the experiments done with Tensorflow 2 and Faster R-CNN for my CSC5651 term project to identify and bound soccer players within live broadcast game footage.

## Loading the model

Let's start by verfying we're utilizing the GPU hardware.

In [2]:
import tensorflow as tf
import json
from IPython.display import Image
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
import pathlib
import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

tf.config.list_physical_devices('GPU')

2023-10-30 17:17:58.404954: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-30 17:17:58.405028: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-30 17:17:58.405110: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-30 17:17:58.423819: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-30 17:18:00.332591: I tensorflow/compiler/

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

Next, we'll load the pre-trained Faster R-CNN model with Inception.

In [3]:
# Download and extract model
def download_model(model_name, model_date):
    base_url = 'http://download.tensorflow.org/models/object_detection/tf2/'
    model_file = model_name + '.tar.gz'
    model_dir = tf.keras.utils.get_file(fname=model_name,
                                        origin=base_url + model_date + '/' + model_file,
                                        untar=True)
    return str(model_dir)

MODEL_DATE = '20200711'
MODEL_NAME = 'faster_rcnn_resnet152_v1_1024x1024_coco17_tpu-8'
PATH_TO_MODEL_DIR = download_model(MODEL_NAME, MODEL_DATE)

In [4]:
PATH_TO_MODEL_DIR

'/home/jack/.keras/datasets/faster_rcnn_resnet152_v1_1024x1024_coco17_tpu-8'

In [5]:
PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR + "/saved_model"

print('Loading model...', end='')
start_time = time.time()

# Load saved model and build the detection function
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

Loading model...

2023-10-30 17:18:06.633314: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2d:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-30 17:18:06.633379: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2d:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-30 17:18:06.633396: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2d:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-30 17:18:07.268808: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2d:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-30 17:18:07.268854: I tensorflow/compile

Done! Took 11.967283248901367 seconds


Load up the labels and create a category index for classification.

In [6]:
def download_labels(filename):
    base_url = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/data/'
    label_dir = tf.keras.utils.get_file(fname=filename,
                                        origin=base_url + filename,
                                        untar=False)
    label_dir = pathlib.Path(label_dir)
    return str(label_dir)

LABEL_FILENAME = 'mscoco_label_map.pbtxt'
PATH_TO_LABELS = download_labels(LABEL_FILENAME)

In [7]:
PATH_TO_LABELS

'/home/jack/.keras/datasets/mscoco_label_map.pbtxt'

In [8]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,
                                                                    use_display_name=True)

## Kicking the tires

Take the model out for a spin by classifying 10 seconds worth of frames. The video is 25 frames/second, so we'll run object detection on 250 frames, draw bounding boxes on them, restructure them back into a video, and output it to a file.

In [114]:
warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings

def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
      path: the file path to the image

    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))

VIDEO_PATH = 'data/issia/filmrole1.avi'
OUTPUT_DIR = '/mnt/d/school/csc5651/soc_output/'
NUM_SECONDS = 10

cap = cv2.VideoCapture(VIDEO_PATH)

fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_file = f'{OUTPUT_DIR}output_video.avi'
fps = 25
frame_size = (1024, 1024)
out = cv2.VideoWriter(output_file, fourcc, fps, frame_size)

i = 0
max_i = 25 * NUM_SECONDS

while i < max_i:
  ret, frame = cap.read()
  resized_frame = cv2.resize(frame, (1024, 1024))

  input_tensor = tf.convert_to_tensor(resized_frame)

  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis, ...]

  # input_tensor = np.expand_dims(image_np, 0)
  detections = detect_fn(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(detections.pop('num_detections'))
  detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
  detections['num_detections'] = num_detections

  # detection_classes should be ints.
  detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

  image = resized_frame.copy()

  viz_utils.visualize_boxes_and_labels_on_image_array(
    image,
    detections['detection_boxes'],
    detections['detection_classes'],
    detections['detection_scores'],
    category_index,
    use_normalized_coordinates=True,
    max_boxes_to_draw=200,
    min_score_thresh=.30,
    agnostic_mode=False)

  out.write(image)
  i += 1

cap.release()
out.release()
cv2.destroyAllWindows()

## Testing the pre-trained model

Now we'll load up the ISSIA dataset with the help of some utility functions borrowed from [FootAndBall](https://github.com/jac99/FootAndBall), a project by Jacek Komorowski et. al. which uses the same dataset for a similar use case.

In [115]:
import issia_utils as iu

# Start with video 1
annotations = iu.read_issia_ground_truth(1, "data/issia/")

Number of frames = 3021


The object `annotations.persons` now contains a dictionary, whose keys are frame numbers and values are lists of tuples containing `(player_id, height, width, x, y)` for each player in the frame.

In [116]:
annotations.persons.get(356)

[('9998', 103, 42, 1385, 320),
 ('9997', 102, 37, 1, 385),
 ('9999', 94, 42, 307, 235)]

For this project, we're only using the player data, so we'll discard the other information for now. If we have time, we'll come back to the ball and test tracking it too. The model we're using has an input size of 1024 x 1024, but the video is a different size. We need to resize the video and the annotations to match.

In [117]:
VIDEO_PATH = 'data/issia/filmrole1.avi'
INPUT_WIDTH = 1024
INPUT_HEIGHT = 1024

cap = cv2.VideoCapture(VIDEO_PATH)

width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

width_scale_factor = INPUT_WIDTH / width
height_scale_factor = INPUT_HEIGHT / height

resized_player_annotations = dict()

for k, v in annotations.persons.items():
  frame = k
  annotation = v
  resized_player_annotations[frame] = list()

  for player in annotation:
    player_id = player[0]
    height = int(player[1] * height_scale_factor)
    width  = int(player[2] * width_scale_factor)
    x = int(player[3] * width_scale_factor)
    y = int(player[4] * height_scale_factor)
    resized_player_annotations[frame].append((player_id, height, width, x, y))

In [119]:
resized_player_annotations.get(356)

[('9998', 96, 22, 738, 301),
 ('9997', 96, 19, 0, 362),
 ('9999', 88, 22, 163, 221)]

This resize operation looks pretty sound. Let's verify by resizing some frames from the video and painting on the boundary boxes to verify.

In [120]:
OUTPUT_DIR = '/mnt/d/school/csc5651/soc_output/'

cap = cv2.VideoCapture(VIDEO_PATH)
fourcc = cv2.VideoWriter_fourcc(*"XVID")
output_file = f"{OUTPUT_DIR}ground_truth_test.avi"
fps = 25
frame_size = (1024, 1024)
out = cv2.VideoWriter(output_file, fourcc, fps, frame_size)

i = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    resized_frame = cv2.resize(frame, (1024, 1024))

    image = resized_frame.copy()

    if i in resized_player_annotations:
        for player in resized_player_annotations.get(i):
            height = player[1]
            width = player[2]
            x = player[3]
            y = player[4]

            cv2.rectangle(image, (x, y), (x + width, y + height), (0, 0, 255), 2)
        out.write(image)

    i += 1

cap.release()
out.release()
cv2.destroyAllWindows()

Viewing back the output footage, the bounding boxes look approximately accurate, which means the algorithm for rescaling the bounding boxes isn't grossly incorrect. We'll continue with these, and resize and trim each video down to only the frames that have ground-truth data associated with them. The first few seconds of frames in each video are devoid of it, as the camera was calibrating during those. While we're at it, let's also dump out the resized annotations to JSON files for easier reading later.

In [121]:
for j in range(1, 7):
    cap = cv2.VideoCapture(f'data/issia/filmrole{j}.avi')
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    output_file = f'{OUTPUT_DIR}resized_filmrole{j}.avi'
    fps = 25
    frame_size = (1024, 1024)
    out = cv2.VideoWriter(output_file, fourcc, fps, frame_size)
    annotations = iu.read_issia_ground_truth(j, "data/issia/")
    resized_player_annotations = dict()
    for k, v in annotations.persons.items():
        frame = k
        annotation = v
        resized_player_annotations[frame] = list()

        for player in annotation:
            player_id = player[0]
            height = int(player[1] * height_scale_factor)
            width = int(player[2] * width_scale_factor)
            x = int(player[3] * width_scale_factor)
            y = int(player[4] * height_scale_factor)
            resized_player_annotations[frame].append((player_id, height, width, x, y))

    i = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        resized_frame = cv2.resize(frame, (1024, 1024))
        out.write(resized_frame)

        i += 1

    with open(f'{OUTPUT_DIR}filmrole{j}.json', 'w') as outfile:
        json.dump(resized_player_annotations, outfile)

    cap.release()
    out.release()
    cv2.destroyAllWindows()

Number of frames = 3021
Number of frames = 3028
Number of frames = 3033
Number of frames = 3024
Number of frames = 3020
Number of frames = 3026


Now that that's done, we can run object detection on one of the resized video clips, and measure the accuracy of the bounding boxes compared to the ground truth. Since we're only looking for persons, we'll filter all the other labels from the detection space of the model.

In [50]:
def filter_detections(detections, label_id, threshold):
    scores = detections['detection_scores'][0].numpy()
    boxes = detections['detection_boxes'][0].numpy()
    classes = detections['detection_classes'][0].numpy().astype(int)

    mask = (classes == label_id) & (scores > threshold)
    scores = scores[mask]
    boxes = boxes[mask] * 1024
    filtered_boxes = list()
    for box in boxes:
        filtered_boxes.append(box)

    return {'detection_scores': scores, 'detection_boxes': np.array(filtered_boxes)}

INPUT_FILENAME = "/mnt/d/school/csc5651/soc_output/resized_filmrole1.avi"
ANNOTATION_FILENAME = "/mnt/d/school/csc5651/soc_output/filmrole1.json"

with open(ANNOTATION_FILENAME) as f:
    ground_truth_annotations = json.load(f)

ground_truth_annotations = {k: v for k, v in ground_truth_annotations.items() if v}

cap = cv2.VideoCapture(INPUT_FILENAME)

i = 0
all_detections = dict()
while cap.isOpened():
    ret, frame = cap.read()
    i += 1
    if not ret:
        break

    if str(i) not in ground_truth_annotations:
        continue

    input_tensor = tf.convert_to_tensor(frame)
    input_tensor = input_tensor[tf.newaxis, ...]
    detections = filter_detections(detect_fn(input_tensor), 1, 0.5)
    all_detections[str(i)] = detections

cap.release()
cv2.destroyAllWindows()

Tensorflow 2 returns detections in the format of `(ymin, xmin, ymax, xmax)`, so we need to convert our ground truth annotations to that format.

In [51]:
def convert_ground_truth_box_to_tf_format(box):
    _, height, width, x, y = box
    ymin = int(y)
    xmin = int(x)
    ymax = int(y + height)
    xmax = int(x + width)
    return np.array([ymin, xmin, ymax, xmax])

converted_ground_truth_annotations = dict()
for k, v in ground_truth_annotations.items():
    converted_boxes = list()
    for box in v:
        converted_boxes.append(convert_ground_truth_box_to_tf_format(box))
    converted_boxes = np.array(converted_boxes)
    converted_ground_truth_annotations[k] = {'detection_boxes': converted_boxes}

Let's do a quick smoke test and compare the detections with the ground truths for one of the frames.

In [64]:
all_detections['356']

{'detection_scores': array([0.7640616 , 0.66895443, 0.5417378 ], dtype=float32),
 'detection_boxes': array([[341.238   ,   0.      , 471.2253  ,  30.268375],
        [303.21304 , 739.10297 , 403.62918 , 761.4393  ],
        [309.772   ,   0.      , 556.49884 ,  29.684807]], dtype=float32)}

In [65]:
converted_ground_truth_annotations['356']

{'detection_boxes': array([[301, 738, 397, 760],
        [362,   0, 458,  19],
        [221, 163, 309, 185]])}

Now that the ground truths and the detections are in the same format, we'll calculate the accuracy and precision frame-by-frame. This algorithm uses _intersection over union_ to determine hits and misses. That is, the area of overlap between the detected box and the ground truth box, the intersection, as a fraction of the total combined area, the union, of both boxes. 0 means no overlap, and 1 means perfect overlap. For this test, we'll call any detection with an IoU value of >= 0.5 a true positive.

In [63]:
def calculate_iou(gt_box, pred_box):
    y1_t, x1_t, y2_t, x2_t = gt_box
    y1_p, x1_p, y2_p, x2_p = pred_box

    if (x1_p > x2_p) or (y1_p > y2_p):
        raise AssertionError(
            "Prediction box is malformed? pred box: {}".format(pred_box))
    if (x1_t > x2_t) or (y1_t > y2_t):
        raise AssertionError(
            "Ground Truth box is malformed? true box: {}".format(gt_box))

    if (x2_t < x1_p or x2_p < x1_t or y2_t < y1_p or y2_p < y1_t):
        return 0.0

    far_x = np.min([x2_t, x2_p])
    near_x = np.max([x1_t, x1_p])
    far_y = np.min([y2_t, y2_p])
    near_y = np.max([y1_t, y1_p])

    inter_area = (far_x - near_x + 1) * (far_y - near_y + 1)
    true_box_area = (x2_t - x1_t + 1) * (y2_t - y1_t + 1)
    pred_box_area = (x2_p - x1_p + 1) * (y2_p - y1_p + 1)
    iou = inter_area / (true_box_area + pred_box_area - inter_area)
    return iou

def calculate_precision_recall(gt_boxes, det_boxes, iou_threshold):
    if det_boxes is None:
        return 0, 0

    # Calculate precision and recall for a set of ground truth and detected bounding boxes
    tp = 0
    fp = 0
    fn = 0

    for det_box in det_boxes:
        max_iou = 0
        max_gt_box = None

        for gt_box in gt_boxes:
            iou = calculate_iou(gt_box, det_box)

            if iou > max_iou:
                max_iou = iou
                max_gt_box = gt_box

        if max_iou >= iou_threshold:
            tp += 1
            gt_boxes = [x for x in gt_boxes if not np.array_equal(x, max_gt_box)]
        else:
            fp += 1

    fn = len(gt_boxes)

    if tp != 0:
        precision = tp / float(tp + fp)
        recall = tp / float(tp + fn)
    else:
        precision = 0
        recall = 0

    return precision, recall

all_precisions = list()
all_recalls = list()
for frame_id in converted_ground_truth_annotations.keys():
    gt_boxes = converted_ground_truth_annotations.get(frame_id).get('detection_boxes')
    det_boxes = all_detections.get(frame_id).get('detection_boxes') if frame_id in all_detections else None
    precision, recall = calculate_precision_recall(gt_boxes, det_boxes, 0.5)
    all_precisions.append(precision)
    all_recalls.append(recall)

print(f'Average precision: {np.mean(all_precisions)}')
print(f'Average recall: {np.mean(all_recalls)}')

Average precision: 0.738977517655109
Average recall: 0.6425215336495264


This first test is about 74% accurate and showed about 64% recall. Let's now combine this with the other 5 videos and ground truth annotations.

In [123]:
INPUT_FILENAME = "/mnt/d/school/csc5651/soc_output/resized_filmrole{}.avi"
ANNOTATION_FILENAME = "/mnt/d/school/csc5651/soc_output/filmrole{}.json"

master_ground_truth_annotations = dict()
master_all_detections = dict()
for i in range(1, 7):
    with open(ANNOTATION_FILENAME.format(i)) as f:
        ground_truth_annotations = json.load(f)

    master_ground_truth_annotations.update({f"{i}_{k}": v for k, v in ground_truth_annotations.items() if v})

    cap = cv2.VideoCapture(INPUT_FILENAME.format(i))

    j = 0
    while cap.isOpened():
        ret, frame = cap.read()
        j += 1
        if not ret:
            break

        if str(j) not in ground_truth_annotations:
            continue

        input_tensor = tf.convert_to_tensor(frame)
        input_tensor = input_tensor[tf.newaxis, ...]
        detections = filter_detections(detect_fn(input_tensor), 1, 0.5)
        master_all_detections[f"{i}_{j}"] = detections

    cap.release()
    cv2.destroyAllWindows()

converted_master_ground_truth_annotations = dict()
for k, v in master_ground_truth_annotations.items():
    converted_boxes = list()
    for box in v:
        converted_boxes.append(convert_ground_truth_box_to_tf_format(box))
    converted_boxes = np.array(converted_boxes)
    converted_master_ground_truth_annotations[k] = {'detection_boxes': converted_boxes}

In [133]:
len(converted_master_ground_truth_annotations)

15992

In [134]:
len(master_all_detections)

15647

In [135]:
all_precisions = list()
all_recalls = list()
for frame_id in converted_master_ground_truth_annotations.keys():
    gt_boxes = converted_master_ground_truth_annotations.get(frame_id).get('detection_boxes')
    det_boxes = master_all_detections.get(frame_id).get('detection_boxes') if frame_id in master_all_detections else None
    precision, recall = calculate_precision_recall(gt_boxes, det_boxes, 0.5)
    all_precisions.append(precision)
    all_recalls.append(recall)

print(f'Average precision over frames in all videos: {np.mean(all_precisions)}')
print(f'Average recall over frames in all videos: {np.mean(all_recalls)}')

Average precision over frames in all videos: 0.6520690548031569
Average recall over frames in all videos: 0.5678544390437077


In [102]:
OUTPUT_DIR = '/mnt/d/school/csc5651/soc_output/'
VIDEO_NUMBER = 3
FRAME_NUMBER = 1001
INPUT_FILENAME = "/mnt/d/school/csc5651/soc_output/resized_filmrole{}.avi".format(VIDEO_NUMBER)

cap = cv2.VideoCapture(INPUT_FILENAME)

i = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    if i != FRAME_NUMBER:
        i += 1
        continue

    resized_frame = cv2.resize(frame, (1024, 1024))

    image = resized_frame.copy()

    for box in converted_master_ground_truth_annotations[f'{VIDEO_NUMBER}_{FRAME_NUMBER}'].get('detection_boxes'):
        ymin, xmin, ymax, xmax = box
        cv2.rectangle(image, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (0, 255, 0), 2)

    for box in master_all_detections[f'{VIDEO_NUMBER}_{FRAME_NUMBER}'].get('detection_boxes'):
        ymin, xmin, ymax, xmax = box
        cv2.rectangle(image, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (0, 0, 255), 2)

    cv2.imwrite(f'{OUTPUT_DIR}{VIDEO_NUMBER}_frame_{FRAME_NUMBER}.jpg', image)
    break

cap.release()
cv2.destroyAllWindows()